In [ ]:
from deep_sort_realtime.deepsort_tracker import DeepSort
import yolov5
import cv2
import numpy
import pandas
import math
import random
from subprocess import run
from datetime import datetime

model = yolov5.load('models/yolov5s.pt')

cap = cv2.VideoCapture("video_data/vid2.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
width = cap. get(cv2. CAP_PROP_FRAME_WIDTH )
height = cap. get(cv2. CAP_PROP_FRAME_HEIGHT )
tracker = DeepSort(max_age=5)
prev = {}
count = 0
fcc = cv2.VideoWriter_fourcc(*'XVID')
result = cv2.VideoWriter('vide_data/output.avi', fcc, 30, (int(width), int(height))) 

l = random.randint(0,255)
j = random.randint(0,255)
k = random.randint(0,255)

track_speed = {}

while True:
    try:
        curr = {}
        ret, frame = cap.read()
        frame = cv2.flip(frame,0)
        if not ret:
            break
        results = model(frame) 
        df = results.pandas().xyxy[0]
        df = df.drop(df[df.ymax < 550].index)
        df = df.drop(df[df.ymax > 650].index)
        df = df.drop(df[df.confidence < 0.4].index)
        df = df.reset_index()

        bbs = []
        for i in range(len(df)) :
            bbs.append( ( [ int(df.xmin[i]) , int(df.ymin[i]) , abs(int(df.xmax[i] - df.xmin[i])) , abs(int(df.ymax[i] - df.ymin[i])) ] , df.confidence[i] , df['class'][i] ) )
            cv2.putText(frame, df.name[i] , (int(df.xmax[i]), int(df.ymax[i]) - 7), 0, 1, (j, l, k), 2)
        
        tracks = tracker.update_tracks(bbs, frame=frame) # bbs expected to be a list of detections, each in tuples of ( [left,top,w,h], confidence, detection_class )
        for track in tracks:
            if not track.is_confirmed():
                continue
            track_id = track.track_id
            ltrb = track.to_ltrb()
            cv2.rectangle(frame, (int(ltrb[0]) , int(ltrb[1])), (int(ltrb[2]) , int(ltrb[3])), (l, k, j), 2)
            cropped_image = frame[ int(ltrb[0]) : int(ltrb[2])  , int(ltrb[1])  : int(ltrb[3])  ]
            f = "C:/Users/Akshit/Desktop/Hackathon/plate_detect/test/img.jpg"
            cv2.imwrite(f, cropped_image)
            !python lpr.py
            file1 = open("plate_detect/number.txt","r") 
            license_plate_number = file1.read()
            cv2.putText(frame, license_plate_number , (int(ltrb[0]) , int(ltrb[3]) + int(ltrb[3]*0.03)  ), 0, 1, (j, l, k), 2)



            if count == 0:
                prev[track_id] = ltrb[:2]
                count += 1
                continue
            else:
                curr[track_id] = ltrb[:2]

        for i in curr.keys():
            try:
                pixel_frame = math.dist( curr[i] , prev[i] ) 
                metre_pixel = 0.05
                frame_second = fps
                speed = pixel_frame * metre_pixel * frame_second * 3.6
                try:
                    track_speed[i].append(speed)
                except :
                    track_speed[i] = [speed]
                sum = 0
                for j in track_speed[i]:
                    sum += j
                avg = sum/len(track_speed[i])
                cv2.putText(frame, str(int(avg)) + " km/hr", (int(curr[i][0]), int(curr[i][1] - 7)), 0, 1, (k, j, l), 2)
            except:
                a = 4
                    
        prev = curr
            
        cv2.imshow("Frame", frame)
        result.write(frame)

        key = cv2.waitKey(1)
        if key == 27:
            break

    except:
        a = 3

result.release()
cap.release()
cv2.destroyAllWindows()